In [ ]:
import pandas as pd

df = pd.read_parquet('../validations/sens_thigh/laboratory.parquet')

df

In [ ]:
import altair as alt
from sklearn.metrics import confusion_matrix
import numpy as np

true = df['ground_truth']
pred = df['activity']
labels = ['lie', 'sit', 'stand', 'walk', 'run', 'bicycle']

if not labels:
    labels = true.unique().tolist() + pred.unique().tolist()
    labels = list(set(labels))

matrix = confusion_matrix(true, pred, labels=labels, normalize='true').round(4) * 100
matrix = np.flip(matrix, axis=1)
matrix = pd.DataFrame(matrix, index=labels, columns=labels[::-1]).reset_index().melt(id_vars='index')
matrix.columns = ['True', 'Predicted', 'Value']

confussion_matrix = (
    alt.Chart(matrix)
    .mark_rect()
    .encode(
        x=alt.X(
            'Predicted:O',
            sort=labels[::-1],
            axis=alt.Axis(
                titleFontSize=14,
                labelFontSize=12,
            ),
        ),
        y=alt.Y(
            'True:O',
            sort=labels,
            axis=alt.Axis(
                titleFontSize=14,
                labelFontSize=12,
            ),
        ),
        color=alt.Color('Value:Q', scale=alt.Scale(scheme='purples'), legend=None),
        tooltip=['True', 'Predicted', 'Value'],
    )
    .properties(title=alt.Title(text='Laboratory', fontSize=16), width=300, height=300)
)

text = confussion_matrix.mark_text(baseline='middle', fontSize=12).encode(
    text=alt.condition(
        alt.datum.Value == 0,
        alt.value(''),  # If the count is 0, display an empty string
        'Value:Q',  # Otherwise, display the count
    ),
    color=alt.condition(alt.datum.Value > 75, alt.value('white'), alt.value('black')),
)
chart = confussion_matrix + text

chart